In [1]:
import numpy as np
import theano
import theano.tensor as TT
import matplotlib.pyplot as plt
import time
from datetime import datetime

# Gradients for RNN on 1 example at a time

## Gradients with simple functions

In [2]:
# Number of input units
nin = 1
# Number of hidden units
nh = 4
# Number of output units
nout = 1

In [3]:
th_W_xh = TT.matrix()
th_W_hh = TT.matrix()
th_W_hy = TT.matrix()

th_x = TT.matrix()
th_y = TT.matrix()
th_h0 = TT.vector()

In [4]:
# Compute gradients using Theano
# Hidden state
#th_h = TT.tanh(TT.dot(th_x, th_W_xh) + TT.dot(th_h0, th_W_hh))
#th_yhat = TT.tanh(TT.dot(th_h, th_W_hy))
def step(th_x_t, th_h_tm1, th_W_xh, th_W_hh, th_W_hy):
    th_h_t = TT.dot(th_x_t, th_W_xh) + TT.dot(th_h_tm1, th_W_hh)
    th_yhat_t = TT.dot(th_h_t, th_W_hy)
    return th_h_t, th_yhat_t

[th_h, th_yhat], _ = theano.scan(step,
                                 sequences=th_x,
                                 outputs_info=[th_h0, None],
                                 non_sequences=[th_W_xh, th_W_hh, 
                                                th_W_hy])

# Gradients using Cost function
th_gW_xh, th_gW_hh, th_gW_hy = \
        TT.grad(0.5 * ((th_yhat - th_y)**2).sum(),
               [th_W_xh, th_W_hh, th_W_hy])
th_fn = theano.function(
        [th_x, th_h0, th_y, th_W_xh, th_W_hh, th_W_hy],
        [th_gW_xh, th_gW_hh, th_gW_hy])

1 #define _CUDA_NDARRAY_C
2 
3 #include <Python.h>
4 #include <structmember.h>
5 #include "theano_mod_helper.h"
6 
7 #include <numpy/arrayobject.h>
8 #include <iostream>
9 
10 #include "cuda_ndarray.cuh"
11 
12 #ifndef CNMEM_DLLEXPORT
13 #define CNMEM_DLLEXPORT
14 #endif
15 
16 #include "cnmem.h"
17 #include "cnmem.cpp"
18 
19 //If true, when there is a gpu malloc or free error, we print the size of allocated memory on the device.
20 #define COMPUTE_GPU_MEM_USED 0
21 
22 //If true, we fill with NAN allocated device memory.
23 #define ALLOC_MEMSET 0
24 
25 //If true, we print out when we free a device pointer, uninitialize a
26 //CudaNdarray, or allocate a device pointer
27 #define PRINT_FREE_MALLOC 0
28 
29 //If true, we do error checking at the start of functions, to make sure there
30 //is not a pre-existing error when the function is called.
31 //You probably need to set the environment variable
32 //CUDA_LAUNCH_BLOCKING=1, and/or modify the CNDA_THREAD_SYNC
33 //preprocessor macro 


['nvcc', '-shared', '-O3', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden', '-Xlinker', '-rpath,/Users/esphinx/.theano/compiledir_Darwin-15.4.0-x86_64-i386-64bit-i386-2.7.11-64/cuda_ndarray', '-I/Users/esphinx/anaconda/lib/python2.7/site-packages/theano/sandbox/cuda', '-I/usr/local/lib/python2.7/site-packages/numpy/core/include', '-I/Users/esphinx/anaconda/include/python2.7', '-I/Users/esphinx/anaconda/lib/python2.7/site-packages/theano/gof', '-o', '/Users/esphinx/.theano/compiledir_Darwin-15.4.0-x86_64-i386-64bit-i386-2.7.11-64/cuda_ndarray/cuda_ndarray.so', 'mod.cu', '-L/Users/esphinx/anaconda/lib', '-lcublas', '-lcudart', '-Xcompiler', '-undefined,dynamic_lookup', '-Xlinker', '-pie']


In [5]:
# Numerical examples
T = 10
np_h0 = np.random.uniform(size=(nh,))
np_x = np.random.uniform(size=(T, nin))
np_y = np.random.uniform(size=(T, nout))

np_W_xh = np.random.uniform(size=(nin, nh))
np_W_hh = np.random.uniform(size=(nh, nh))
np_W_hy = np.random.uniform(size=(nh, nout))

In [6]:
# Compute gradients using Numpy
#np_h = np.tanh(np.dot(np_x, np_W_xh) + np.dot(np_h0, np_W_hh))
#np_y = np.tanh(np.dot(np_h, np_W_hy))

# Forward propagation
def forwardPropagation(np_x, np_h0, W_xh, W_hh, W_hy):
    h_tm1 = np_h0
    T = np_x.shape[0]
    nin, nh = W_xh.shape
    nout = W_hy.shape[1]
    np_h = np.zeros((T+1, nh))
    np_h[0,:] = np_h0
    np_yhat = np.zeros((T+1, nout))
    for t in xrange(1, T+1):
        h_t = np.dot(np_x[t-1], W_xh) + np.dot(h_tm1, W_hh)
        np_h[t, :] = h_t
        h_tm1 = h_t
        yhat = np.dot(h_t, W_hy)
        np_yhat[t, :] = yhat
    return np_h, np_yhat
    
np_h, np_yhat = forwardPropagation(np_x, np_h0, np_W_xh, np_W_hh, np_W_hy)
loss = 0.5 * ((np_yhat[1:,:] - np_y)**2).sum()

In [7]:
# Theano gradients
th_dE_xh, th_dE_hh, th_dE_hy = \
    th_fn(np_x, np_h0, np_y, np_W_xh, np_W_hh, np_W_hy)
print "th_dE_xh:"
print th_dE_xh
print "th_dE_hh:"
print th_dE_hh
print "th_dE_hy:"
print th_dE_hy

th_dE_xh:
[[ 107124.3642154    66946.95200285   87496.67318075   56405.26129492]]
th_dE_hh:
[[ 869269.99195692  547150.25500308  721317.96648808  457142.57060317]
 [ 935828.73067963  589457.89111612  777747.07216035  492084.3838109 ]
 [ 283882.65488322  178320.86886155  234487.71733382  149348.00445349]
 [ 653380.1672786   411334.08543318  542387.71432151  343596.10833849]]
th_dE_hy:
[[ 191627.64265774]
 [ 225980.52998534]
 [  45158.0464313 ]
 [ 147792.27719617]]


In [8]:
# Numpy gradients
# np_h : T  x nh, np_yhat : T x nout
np_dE_hy = np.dot(np_h[1:,:].T, (np_yhat[1:, :] - np_y))
print "np_dE_hy:"
print np_dE_hy

np_dE_hy:
[[ 191627.64265774]
 [ 225980.52998534]
 [  45158.0464313 ]
 [ 147792.27719617]]


In [9]:
dE_hh = np.zeros((nh, nh))

dhh = np.zeros((T+1, nh))
for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    dhh[t] += np.dot(np_W_hy, (np_yhat[t,:] - np_y[t-1,:]))
    dhh[t-1] += np.dot(np_W_hh, dhh[t])
    dE_hh += np.outer(np_h[t-1,:], dhh[t])
print "dE_hh:"
print dE_hh
print "th_dE_hh:"
print th_dE_hh
print np.allclose(dE_hh, th_dE_hh)

dE_hh:
[[ 869269.99195692  547150.25500308  721317.96648808  457142.57060317]
 [ 935828.73067963  589457.89111612  777747.07216035  492084.3838109 ]
 [ 283882.65488322  178320.86886155  234487.71733382  149348.00445349]
 [ 653380.1672786   411334.08543318  542387.71432151  343596.10833849]]
th_dE_hh:
[[ 869269.99195692  547150.25500308  721317.96648808  457142.57060317]
 [ 935828.73067963  589457.89111612  777747.07216035  492084.3838109 ]
 [ 283882.65488322  178320.86886155  234487.71733382  149348.00445349]
 [ 653380.1672786   411334.08543318  542387.71432151  343596.10833849]]
True


In [10]:
dE_xh = np.zeros((nin, nh))

dhh = np.zeros((T+1, nh))
for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    dhh[t] += np.dot(np_W_hy, (np_yhat[t,:] - np_y[t-1,:]))
    dhh[t-1] += np.dot(np_W_hh, dhh[t])
    dE_xh += np.outer(np_x[t-1,:], dhh[t])
print "dE_xh:"
print dE_xh
print "th_dE_xh:"
print th_dE_xh
print np.allclose(dE_xh, th_dE_xh)

dE_xh:
[[ 107124.3642154    66946.95200285   87496.67318075   56405.26129492]]
th_dE_xh:
[[ 107124.3642154    66946.95200285   87496.67318075   56405.26129492]]
True


## Gradients with tanh non-linearity

In [11]:
# Compute gradients using Theano
# Hidden state
#th_h = TT.tanh(TT.dot(th_x, th_W_xh) + TT.dot(th_h0, th_W_hh))
#th_yhat = TT.tanh(TT.dot(th_h, th_W_hy))
def step(th_x_t, th_h_tm1, th_W_xh, th_W_hh, th_W_hy):
    th_h_t = TT.tanh(TT.dot(th_x_t, th_W_xh) + TT.dot(th_h_tm1, th_W_hh))
    th_yhat_t = TT.tanh(TT.dot(th_h_t, th_W_hy))
    return th_h_t, th_yhat_t

[th_h, th_yhat], _ = theano.scan(step,
                                 sequences=th_x,
                                 outputs_info=[th_h0, None],
                                 non_sequences=[th_W_xh, th_W_hh, 
                                                th_W_hy])

# Gradients using Cost function
th_gW_xh, th_gW_hh, th_gW_hy = \
        TT.grad(0.5 * ((th_yhat - th_y)**2).sum(),
               [th_W_xh, th_W_hh, th_W_hy])
th_fn = theano.function(
        [th_x, th_h0, th_y, th_W_xh, th_W_hh, th_W_hy],
        [th_gW_xh, th_gW_hh, th_gW_hy])

In [12]:
# Numerical examples
T = 10
np_h0 = np.random.uniform(size=(nh,))
np_x = np.random.uniform(size=(T, nin))
np_y = np.random.uniform(size=(T, nout))

np_W_xh = np.random.uniform(size=(nin, nh))
np_W_hh = np.random.uniform(size=(nh, nh))
np_W_hy = np.random.uniform(size=(nh, nout))

In [13]:
# Compute gradients using Numpy
#np_h = np.tanh(np.dot(np_x, np_W_xh) + np.dot(np_h0, np_W_hh))
#np_y = np.tanh(np.dot(np_h, np_W_hy))

# Forward propagation
def forwardPropagation(np_x, np_h0, W_xh, W_hh, W_hy):
    h_tm1 = np_h0
    T = np_x.shape[0]
    nin, nh = W_xh.shape
    nout = W_hy.shape[1]
    np_h = np.zeros((T+1, nh))
    np_h[0,:] = np_h0
    np_yhat = np.zeros((T+1, nout))
    for t in xrange(1, T+1):
        h_t = np.tanh(np.dot(np_x[t-1], W_xh) + np.dot(h_tm1, W_hh))
        np_h[t, :] = h_t
        h_tm1 = h_t
        yhat = np.tanh(np.dot(h_t, W_hy))
        np_yhat[t, :] = yhat
    return np_h, np_yhat
    
np_h, np_yhat = forwardPropagation(np_x, np_h0, np_W_xh, np_W_hh, np_W_hy)
loss = 0.5 * ((np_yhat[1:,:] - np_y)**2).sum()

In [14]:
# Theano gradients
th_dE_xh, th_dE_hh, th_dE_hy = \
    th_fn(np_x, np_h0, np_y, np_W_xh, np_W_hh, np_W_hy)
print "th_dE_xh:"
print th_dE_xh
print "th_dE_hh:"
print th_dE_hh
print "th_dE_hy:"
print th_dE_hy

th_dE_xh:
[[ 0.00010295  0.00259489  0.04856052  0.00426662]]
th_dE_hh:
[[ 0.00029612  0.00626318  0.09047982  0.00933075]
 [ 0.00028583  0.00608747  0.08944656  0.00915609]
 [ 0.00025834  0.00546198  0.07707065  0.008035  ]
 [ 0.00023314  0.00517132  0.08216408  0.00814138]]
th_dE_hy:
[[ 0.43382863]
 [ 0.43207385]
 [ 0.36957455]
 [ 0.41207627]]


In [15]:
# Numpy gradients
# np_h : T  x nh, np_yhat : T x nout
np_dE_hy = np.dot(np_h[1:,:].T, 
          (np_yhat[1:, :] - np_y) * (1 - np_yhat[1:, :] ** 2))
print "np_dE_hy:"
print np_dE_hy
print "th_dE_hy:"
print th_dE_hy
print np.allclose(np_dE_hy, th_dE_hy)

np_dE_hy:
[[ 0.43382863]
 [ 0.43207385]
 [ 0.36957455]
 [ 0.41207627]]
th_dE_hy:
[[ 0.43382863]
 [ 0.43207385]
 [ 0.36957455]
 [ 0.41207627]]
True


In [16]:
dE_hh = np.zeros((nh, nh))

dhh = np.zeros((T+1, nh))
for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    dhh[t] += np.dot(np_W_hy, 
             (np_yhat[t,:] - np_y[t-1,:]) * (1 - np_yhat[t,:] ** 2))
    r = dhh[t] * (1 - np_h[t,:] ** 2)
    dhh[t-1] += np.dot(np_W_hh, r)
    dE_hh += np.outer(np_h[t-1,:], r)
print "dE_hh:"
print dE_hh
print "th_dE_hh:"
print th_dE_hh
print np.allclose(dE_hh, th_dE_hh)

dE_hh:
[[ 0.00029612  0.00626318  0.09047982  0.00933075]
 [ 0.00028583  0.00608747  0.08944656  0.00915609]
 [ 0.00025834  0.00546198  0.07707065  0.008035  ]
 [ 0.00023314  0.00517132  0.08216408  0.00814138]]
th_dE_hh:
[[ 0.00029612  0.00626318  0.09047982  0.00933075]
 [ 0.00028583  0.00608747  0.08944656  0.00915609]
 [ 0.00025834  0.00546198  0.07707065  0.008035  ]
 [ 0.00023314  0.00517132  0.08216408  0.00814138]]
True


In [17]:
dE_xh = np.zeros((nin, nh))

dhh = np.zeros((T+1, nh))
for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    dhh[t] += np.dot(np_W_hy, 
             (np_yhat[t,:] - np_y[t-1,:]) * (1 - np_yhat[t,:] ** 2))
    r = dhh[t] * (1 - np_h[t,:] ** 2)
    dhh[t-1] += np.dot(np_W_hh, r)
    #dE_xh += np.outer(np_x[t-1,:], r)
    dE_xh += np.reshape(np_x[t-1,:], (np_x.shape[1], 1)) * r.T
print "dE_xh:"
print dE_xh
print "th_dE_xh:"
print th_dE_xh
print np.allclose(dE_xh, th_dE_xh)


dE_xh:
[[ 0.00010295  0.00259489  0.04856052  0.00426662]]
th_dE_xh:
[[ 0.00010295  0.00259489  0.04856052  0.00426662]]
True


# Gradients for RNN with mini-batches

In [18]:
# Number of input units
nin = 3
# Number of hidden units
nh = 7
# Number of output units
nout = 1
# Number of batches
nbatches = 10

In [19]:
th_W_xh = TT.matrix()
th_W_hh = TT.matrix()
th_W_hy = TT.matrix()

th_x = TT.tensor3()
th_y = TT.tensor3()
th_h0 = TT.matrix()

In [20]:
# Compute gradients using Theano
# Hidden state
#th_h = TT.tanh(TT.dot(th_x, th_W_xh) + TT.dot(th_h0, th_W_hh))
#th_yhat = TT.tanh(TT.dot(th_h, th_W_hy))
def step(th_x_t, th_h_tm1, th_W_xh, th_W_hh, th_W_hy):
    th_h_t = TT.tanh(TT.dot(th_x_t, th_W_xh) + TT.dot(th_h_tm1, th_W_hh))
    th_yhat_t = TT.tanh(TT.dot(th_h_t, th_W_hy))
    return th_h_t, th_yhat_t

[th_h, th_yhat], _ = theano.scan(step,
                                 sequences=th_x,
                                 outputs_info=[th_h0, None],
                                 non_sequences=[th_W_xh, th_W_hh, 
                                                th_W_hy])

th_error = 0.5 * ((th_yhat - th_y) ** 2).sum().sum()
# Gradients using Cost function
th_gW_xh, th_gW_hh, th_gW_hy = \
                    TT.grad(th_error, [th_W_xh, th_W_hh, th_W_hy])
th_fn = theano.function(
        [th_x, th_h0, th_y, th_W_xh, th_W_hh, th_W_hy],
        [th_gW_xh, th_gW_hh, th_gW_hy])

In [21]:
# Numerical examples
T = 10
np_h0 = np.random.uniform(size=(nbatches, nh))
np_x = np.random.uniform(size=(T, nbatches, nin))
np_y = np.random.uniform(size=(T, nbatches, nout))

np_W_xh = np.random.uniform(size=(nin, nh))
np_W_hh = np.random.uniform(size=(nh, nh))
np_W_hy = np.random.uniform(size=(nh, nout))

In [22]:
# Theano gradients
th_dE_xh, th_dE_hh, th_dE_hy = \
    th_fn(np_x, np_h0, np_y, np_W_xh, np_W_hh, np_W_hy)
print "th_dE_xh:"
print th_dE_xh
print "th_dE_hh:"
print th_dE_hh
print "th_dE_hy:"
print th_dE_hy

th_dE_xh:
[[ 0.0039954   0.00416409  0.00063727  0.00010572  0.00279952  0.01228623
   0.01305023]
 [ 0.012443    0.0111805   0.00122589  0.00054987  0.00638779  0.02178284
   0.01558321]
 [ 0.00693897  0.00648802  0.00094858  0.00025519  0.00413016  0.01322138
   0.01588981]]
th_dE_hh:
[[  6.63339083e-03   8.55404530e-03   1.31970141e-03   1.96182336e-04
    5.42491988e-03   1.65474948e-02   2.34291788e-02]
 [  3.69880987e-03   5.96005918e-03   1.03906158e-03   8.01001790e-05
    4.76751013e-03   1.10643960e-02   1.99483112e-02]
 [  6.01472254e-03   8.13967509e-03   1.28406162e-03   1.71797797e-04
    5.50544715e-03   1.54187719e-02   2.25018397e-02]
 [  3.66507301e-03   5.96895792e-03   1.07327038e-03   6.25557291e-05
    4.87360108e-03   1.05508715e-02   2.10190710e-02]
 [  6.35283641e-03   8.56755202e-03   1.32520573e-03   1.98669478e-04
    5.92387373e-03   1.47818460e-02   2.14457156e-02]
 [  1.00808944e-02   1.07176140e-02   1.38939915e-03   4.23266838e-04
    6.56216960e-03   1

In [23]:
# Compute gradients using Numpy
# Forward propagation
def forwardPropagationBatch(np_x, np_h0, W_xh, W_hh, W_hy):
    h_tm1 = np_h0
    T = np_x.shape[0]
    nin, nh = W_xh.shape
    nout = W_hy.shape[1]
    np_h = np.zeros((T+1, nbatches, nh))
    np_h[0,:, :] = np_h0
    np_yhat = np.zeros((T+1, nbatches, nout))
    for t in xrange(1, T+1):
        h_t = np.tanh(np.dot(np_x[t-1], W_xh) + np.dot(h_tm1, W_hh))
        np_h[t, :, :] = h_t
        h_tm1 = h_t
        yhat = np.tanh(np.dot(h_t, W_hy))
        np_yhat[t, :, :] = yhat
    return np_h, np_yhat
    
np_h, np_yhat = forwardPropagationBatch(np_x, np_h0, 
                                   np_W_xh, np_W_hh, np_W_hy)
loss = 0.5 * ((np_yhat[1:,:,:] - np_y)**2).sum().sum()
"""
print "np_h:"
print np_h
print "np_yhat:"
print np_yhat
"""
snp_h = np.zeros((T+1, nbatches, nh))
snp_yhat = np.zeros((T+1, nbatches, nout))
for nb in xrange(nbatches):
    cur_h, cur_yhat = \
    forwardPropagation(np_x[:, nb, :], np_h0[nb, :], 
                       np_W_xh, np_W_hh, np_W_hy)
    snp_h[:, nb, :] = cur_h
    snp_yhat[:, nb, :] = cur_yhat
"""
print "snp_h:"
print snp_h
print "snp_yhat:"
print snp_yhat
"""
print np.allclose(np_h, snp_h)
print np.allclose(np_yhat, snp_yhat)

True
True


In [24]:
# Numpy gradients
# np_h : T  x nh, np_yhat : T x nout

h1 = np_h[1:,:,:]
print h1.shape
y1 = (np_yhat[1:, :, :] - np_y) * (1 - np_yhat[1:, :, :] ** 2)
print y1.shape
np_dE_hy = np.tensordot(h1, y1, axes=([0, 1], [0, 1]))
print "np_dE_hy:"
print np_dE_hy
print "th_dE_hy:"
print th_dE_hy
print np.allclose(np_dE_hy, th_dE_hy)

(10, 10, 7)
(10, 10, 1)
np_dE_hy:
[[ 6.27329127]
 [ 6.23557627]
 [ 6.25969545]
 [ 6.28069559]
 [ 6.19100554]
 [ 6.26275851]
 [ 6.25058935]]
th_dE_hy:
[[ 6.27329127]
 [ 6.23557627]
 [ 6.25969545]
 [ 6.28069559]
 [ 6.19100554]
 [ 6.26275851]
 [ 6.25058935]]
True


In [25]:
dE_hh = np.zeros((nh, nh))

for nb in xrange(nbatches):
    dhh = np.zeros((T+1, nh))
    for t in xrange(T,0,-1):
        # dE_t/dy_t * dy_t/dh_t
        dhh[t] += np.dot(np_W_hy, 
                 (np_yhat[t,nb,:] - np_y[t-1,nb,:]) * (1 - np_yhat[t,nb,:] ** 2))
        r = dhh[t] * (1 - np_h[t,nb,:] ** 2)
        dhh[t-1] += np.dot(np_W_hh, r)
        #dE_hh += np.outer(np_h[t-1,nb,:], r)
        dE_hh += np.reshape(np_h[t - 1, nb, :], (nh, 1)) * r
        
print "dE_hh:"
print dE_hh
print "th_dE_hh:"
print th_dE_hh
print np.allclose(dE_hh, th_dE_hh)

dE_hh = np.zeros((nh, nh))
dhh = np.zeros((T+1, nbatches, nh))
for t in xrange(T, 0, -1):
    for nb in xrange(nbatches):
        dhh[t, nb, :] += \
            np.dot(np_W_hy,
            (np_yhat[t,nb,:] - np_y[t-1,nb,:]) * (1 - np_yhat[t,nb,:] ** 2))
        r = dhh[t, nb, :] * (1 - np_h[t, nb, :] ** 2)
        dhh[t-1, nb, :] += np.dot(np_W_hh, r)
        dE_hh += np.reshape(np_h[t-1, nb, :], (nh,1)) * r

print "dE_hh:"
print dE_hh
print "th_dE_hh:"
print th_dE_hh
print np.allclose(dE_hh, th_dE_hh)

dE_hh:
[[  6.63339083e-03   8.55404530e-03   1.31970141e-03   1.96182336e-04
    5.42491988e-03   1.65474948e-02   2.34291788e-02]
 [  3.69880987e-03   5.96005918e-03   1.03906158e-03   8.01001790e-05
    4.76751013e-03   1.10643960e-02   1.99483112e-02]
 [  6.01472254e-03   8.13967509e-03   1.28406162e-03   1.71797797e-04
    5.50544715e-03   1.54187719e-02   2.25018397e-02]
 [  3.66507301e-03   5.96895792e-03   1.07327038e-03   6.25557291e-05
    4.87360108e-03   1.05508715e-02   2.10190710e-02]
 [  6.35283641e-03   8.56755202e-03   1.32520573e-03   1.98669478e-04
    5.92387373e-03   1.47818460e-02   2.14457156e-02]
 [  1.00808944e-02   1.07176140e-02   1.38939915e-03   4.23266838e-04
    6.56216960e-03   1.87813611e-02   2.09511603e-02]
 [  8.01225237e-03   9.30569074e-03   1.23629506e-03   3.24521142e-04
    6.13618933e-03   1.56907242e-02   1.92055574e-02]]
th_dE_hh:
[[  6.63339083e-03   8.55404530e-03   1.31970141e-03   1.96182336e-04
    5.42491988e-03   1.65474948e-02   2.3429

In [26]:
dE_hh = np.zeros((nh, nh))
dhh = np.zeros((T+1, nbatches, nh))

for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    m = (np_yhat[t, :, :] - np_y[t-1, :, :]) * (1 - np_yhat[t, :, :] ** 2)
    dhh[t, :, :] += np.dot(m, np_W_hy.T)
    r = dhh[t, :, :] * (1 - np_h[t, :, :] ** 2)
    
    dhh[t-1, :, :] += np.dot(r, np_W_hh.T)
    dE_hh += np.dot(np_h[t-1, :, :].T, r)
    
print "dE_hh:"
print dE_hh
print "th_dE_hh:"
print th_dE_hh
print np.allclose(dE_hh, th_dE_hh)

dE_hh:
[[  6.63339083e-03   8.55404530e-03   1.31970141e-03   1.96182336e-04
    5.42491988e-03   1.65474948e-02   2.34291788e-02]
 [  3.69880987e-03   5.96005918e-03   1.03906158e-03   8.01001790e-05
    4.76751013e-03   1.10643960e-02   1.99483112e-02]
 [  6.01472254e-03   8.13967509e-03   1.28406162e-03   1.71797797e-04
    5.50544715e-03   1.54187719e-02   2.25018397e-02]
 [  3.66507301e-03   5.96895792e-03   1.07327038e-03   6.25557291e-05
    4.87360108e-03   1.05508715e-02   2.10190710e-02]
 [  6.35283641e-03   8.56755202e-03   1.32520573e-03   1.98669478e-04
    5.92387373e-03   1.47818460e-02   2.14457156e-02]
 [  1.00808944e-02   1.07176140e-02   1.38939915e-03   4.23266838e-04
    6.56216960e-03   1.87813611e-02   2.09511603e-02]
 [  8.01225237e-03   9.30569074e-03   1.23629506e-03   3.24521142e-04
    6.13618933e-03   1.56907242e-02   1.92055574e-02]]
th_dE_hh:
[[  6.63339083e-03   8.55404530e-03   1.31970141e-03   1.96182336e-04
    5.42491988e-03   1.65474948e-02   2.3429

In [27]:
dE_xh = np.zeros((nin, nh))
dhh = np.zeros((T+1, nbatches, nh))

for t in xrange(T,0,-1):
    # dE_t/dy_t * dy_t/dh_t
    m = (np_yhat[t, :, :] - np_y[t-1, :, :]) * (1 - np_yhat[t, :, :] ** 2)
    dhh[t, :, :] += np.dot(m, np_W_hy.T)
    r = dhh[t, :, :] * (1 - np_h[t, :, :] ** 2)
    
    dhh[t-1, :, :] += np.dot(r, np_W_hh.T)
    dE_xh += np.dot(np_x[t-1, :, :].T, r)
    
print "dE_xh:"
print dE_xh
print "th_dE_xh:"
print th_dE_xh
print np.allclose(dE_xh, th_dE_xh)

dE_xh:
[[ 0.0039954   0.00416409  0.00063727  0.00010572  0.00279952  0.01228623
   0.01305023]
 [ 0.012443    0.0111805   0.00122589  0.00054987  0.00638779  0.02178284
   0.01558321]
 [ 0.00693897  0.00648802  0.00094858  0.00025519  0.00413016  0.01322138
   0.01588981]]
th_dE_xh:
[[ 0.0039954   0.00416409  0.00063727  0.00010572  0.00279952  0.01228623
   0.01305023]
 [ 0.012443    0.0111805   0.00122589  0.00054987  0.00638779  0.02178284
   0.01558321]
 [ 0.00693897  0.00648802  0.00094858  0.00025519  0.00413016  0.01322138
   0.01588981]]
True
